This is my general scratchwork/ML Tinkering space to become more familiar with Symbollic Regression and the data we will be working with for this project

In [ ]:
#Imports Below
import pandas as pd
import numpy as np
import operator
import random
import io

import math
!pip install gplearn
from gplearn.genetic import SymbolicRegressor
from sympy import sympify, simplify
import matplotlib

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn import metrics

#!pip install google-colab
from google.colab import files
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [ ]:
#Uploading Datasets
IntxnStr=pd.read_csv('/content/drive/Shareddrives/Capstone Drive/MLTesting/Datasets/ExpPatch_IntxnStr.csv')
PatchTidalHeights=pd.read_csv('/content/drive/Shareddrives/Capstone Drive/MLTesting/Datasets/ExpPatch_PatchTidalHeights.csv')
PhotoCounts=pd.read_csv('/content/drive/Shareddrives/Capstone Drive/MLTesting/Datasets/ExpPatch_PhotoCounts.csv')
QuadLocations=pd.read_csv('/content/drive/Shareddrives/Capstone Drive/MLTesting/Datasets/ExpPatch_QuadLocations.csv')
TempsCleaned=pd.read_csv('/content/drive/Shareddrives/Capstone Drive/MLTesting/Datasets/ExpPatch_TempsCleaned.csv')
BalanusGrowthRate=pd.read_csv('/content/drive/Shareddrives/Capstone Drive/MLTesting/Datasets/BalanusGrowthRate.csv')


In [ ]:
#View Data
IntxnStr.head()
#IntxnStr.describe()

,Date,Patch,Prey,Pred,frat,fcnt,tObs,htime,Density,avgTemp,att,frate
0,2013-08-19,A,Adula_californiensis,Nucella_canaliculata,0.0,0,1,NaN,0.0,12.22,NaN,NaN
1,2013-08-19,A,Adula_californiensis,Nucella_ostrina,0.0,0,36,NaN,0.0,12.22,NaN,NaN
2,2013-08-19,A,Balanus_glandula,Nucella_canaliculata,0.0,0,1,NaN,101683.2,12.22,NaN,NaN
3,2013-08-19,A,Balanus_glandula,Nucella_ostrina,0.2,6,36,13.338241,101683.2,12.22,1.474627e-07,0.014994
4,2013-08-19,A,Chthamalus_dalli,Nucella_canaliculata,0.0,0,1,NaN,819.2,12.22,NaN,NaN


In [ ]:
converter = {
    'sub': lambda x, y : x - y,
    'div': lambda x, y : x/y,
    'mul': lambda x, y : x*y,
    'add': lambda x, y : x + y,
    'neg': lambda x    : -x,
    'pow': lambda x, y : x**y,
    'inv': lambda x: 1/x,
    'sqrt': lambda x: x**0.5,
    'pow3': lambda x: x**3,
    'sin': lambda x: math.sin(x),
    'cos': lambda x: math.cos(x),
}

In [ ]:
#isolating Balanus_glandula and modifying dataset to run cleaner
preyItems = ['Balanus_glandula']
df_working_s1 = IntxnStr
#Isolate Nucella_ostrina as the prey
df_working_s2 = df_working_s1[df_working_s1.Pred=='Nucella_ostrina']
#df_working_s2.frate.isna().sum() #199 nas for feeding rate out of 486
#remove na's
df_working_s3 = df_working_s2[df_working_s2['frate'].notna()]
#remove many of the columns
df_working_s4 = df_working_s3.filter(['frat','fcnt','tObs','htime','Density','avgTemp','att','frate'])
#remove infinites
df_working_s5 = df_working_s4.replace([np.inf, 0], np.nan).dropna(axis=0)
#remove outliers
df_working_s6 = df_working_s5[df_working_s5['frate'] < .060]
df_working_s6

In [ ]:
#grabbing X_train and y_train
function_set = ['add', 'sub', 'mul', 'div'] #the functions we'll be using
y_train=df_working_s6.filter(['frate'],axis=1)
X_train=df_working_s6.filter(['Density','avgTemp'],axis=1) #just include feeding rate and temp
full=df_working_s5.filter(['Density','avgTemp','frate'],axis=1) #for plotting and seeing outliers

#check to make sure all is well
print (X_train)
#print (X_train.isna().sum().sum())
#print (X_train.shape)
print (y_train)
y_train.describe()

In [ ]:
#visualize outliers
import plotly.express as px
full.plot(kind='scatter', x='Density', y='frate')
full.plot(kind='scatter', x='avgTemp', y='frate')
temp=full[full['frate'] < .060]
temp.plot(kind='scatter', x='Density', y='frate')

fig = px.scatter_3d(temp, x='Density', y='avgTemp', z='frate')
fig.show()




In [ ]:
#time to run a grid search for good parameters
#this cell can be skipped unless you want to run a grid search
#function_set = ['add', 'sub', 'mul', 'div', 'neg','inv','cos','sin','log','sqrt']
feature_names_list=['Density','avgTemp']

param_population_size=[4000]#---------------default 1k
#param_generations=[10]#---------------------default 20
param_tournament_size=[80]#----------------default 20, selects 20 from population size to find winners
#param_stopping_critera=[.000001]#-----------default 0.0
#param_function_set=[function_set]#----------default (‘add’, ‘sub’, ‘mul’, ‘div’)
param_p_crossover=[.83,.87,.9,.95,.97]#-----------default .9
param_p_subtree_mutation=[.01,.03,.05,.1]#--default .01
param_p_hoist_mutation=[.01,.03,.05,.1]#----default .01
param_p_point_mutation=[.01,.03,.05,.1]#----default .01 
#param_max_samples=[1.0]#--------------------default 1
#param_parsimony_coefficient=[.000001]#------default .0001
#param_low_memory=['True']#------------------default false

param_complete=[{'population_size':param_population_size,
                 #'generations':param_generations,
                 #'stopping_criteria':param_stopping_critera,
                 'p_crossover':param_p_crossover,
                 'p_subtree_mutation':param_p_subtree_mutation,
                 #'function_set':param_function_set,
                 'p_hoist_mutation':param_p_hoist_mutation,
                 'p_point_mutation':param_p_point_mutation,
                 #'max_samples':param_max_samples,
                 #'parsimony_coefficient':param_parsimony_coefficient,
                 #'n_jobs':[-1],
                 #'feature_names':[feature_names_list],
                 #'low_memory':param_low_memory,
                 'tournament_size':param_tournament_size,
                 #'random_state':[0],
                 #'verbose':[1],
                 }]

In [ ]:
#grid search
#this cell can be skipped unless you want to run a grid search
clf=SymbolicRegressor(function_set=function_set,
max_samples=.9, verbose=0, generations=15,
parsimony_coefficient=.000002,low_memory='True')

#cv = RepeatedKFold(n_splits=2, n_repeats=1, random_state=0)
gs=GridSearchCV(clf,param_complete,cv=2,verbose=3,scoring='neg_mean_absolute_error')
gs=gs.fit(X_train,y_train.values.ravel())
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
#regressor Warm_Start not working?
#the meat'n'potatoes 
est_gp = SymbolicRegressor(population_size=5000, 
generations=10, stopping_criteria=.0015, 
p_crossover=0.9, p_subtree_mutation=0.01, function_set=function_set,
p_hoist_mutation=0.05, 
p_point_mutation=0.01,
max_samples=.8, verbose=1, tournament_size=100,
parsimony_coefficient=.000002, warm_start='True',
low_memory='True')

est_gp.fit(X_train, y_train.values.ravel())
print(est_gp._program)
print('R2:',est_gp.score(X_train,y_train))


Some generated equations

div(-0.233, add(add(add(add(mul(X1, X1), div(X1, -0.041)), div(-0.233, -0.041)), div(-0.233, div(add(add(add(add(add(mul(X1, X1), div(X1, -0.041)), div(add(add(add(add(add(add(mul(X1, X1), div(X1, -0.041)), div(div(-0.041, X0), X0)), div(div(div(X1, -0.041), X0), X0)), div(-0.233, div(mul(X1, X1), X0))), div(-0.233, div(div(X1, -0.041), X0))), X1), X0)), div(-0.233, div(mul(X1, X1), X0))), div(-0.233, div(div(X1, -0.041), X0))), X1), X0))), X1))
R2: 0.15202983834837858

add(mul(0.362, mul(0.060, 0.042)), mul(mul(mul(mul(0.060, 0.042), 0.042), -0.076), mul(sub(mul(0.060, 0.042), mul(0.362, X0)), 0.042)))
R2: 0.20055316483284125

div(mul(add(div(mul(mul(mul(-0.012, X1), X0), add(add(mul(add(add(X0, X0), add(X0, X0)), sub(X0, mul(add(X0, X0), mul(0.072, X1)))), add(X0, X0)), div(div(add(X0, X0), -0.012), -0.012))), div(div(X0, -0.012), -0.012)), X0), mul(-0.012, X1)), div(X0, -0.012))
R2: 0.2701470697263707

In [ ]:
#clean up equation
eq= est_gp._program
r2= est_gp.score(X_train,y_train)
print (r2)
print (eq)
next_e = sympify(eq, locals=converter)
print(next_e)

In [ ]:
#Create tree graph of equation
import graphviz
dot_data = est_gp._program.export_graphviz()
graph = graphviz.Source(dot_data)
graph.render('images/ex1', format='png', cleanup=True)
graph